In [35]:
!pip install googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40716 sha256=126e658915610d5b638039f20a7789a62489c88b1a546a81914c2cd239edd1a2
  Stored in directory: /Users/shawn/Library/Caches/pip/wheels/d9/5f/46/54a2bdb4bcb07d3faba4463d2884865705914cc72a7b8bb5f0
Successfully built googlemaps


In [3]:
from serpapi import GoogleSearch
from urllib.parse import urlsplit, parse_qsl
import os, json
from collections import defaultdict
import numpy as np
import pandas as pd
from utils import *
import googlemaps as gmaps
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import datetime

In [1]:
serp_api_key = "3b756770793fe4a71bd041d460d9399f10fa1f7e15381d13e25732fd2aba9a86"
api_key = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

- name: the name of station
- business_status: 'OPERATIONAL', 'CLOSED_PERMANENTLY', 'CLOSED_TEMPORARILY'
- address: address of charging station
- lat: latitude, 5 after decimal point
- lng: longitude, 5 after decimal point
- types: 'point_of_interest, establishment', 'gas_station, point_of_interest, establishment'
- user_ratings_total: aggregate result (sum) of all users' review to that station
- id: charge_id
- author_name: name of author who wrote the comment
- author_url: the url to that comment
- Rating: rating by this comment
- Time: time of posting that comment
- Relative_time_description: redundant feature to time, for example, 6 months ago, 1 year ago. 
- Text: the content of comment


In [4]:
# Define the client
client = gmaps.Client(api_key)

# Define the area for the search
location = '37.7749,-122.4194' # San Francisco
#location = 'Livonia, MI'
radius = 50 # In meters

# Use the textsearch function to get a list of charging stations
query = 'electric vehicle charging station'
results = client.places(query=query, location=location, radius=radius)

# Create a list to store the overview and review data
data = []

# Loop through each charging station
for place in results['results']:
    # Check if the charging station has reviews

    # Get the place ID for the charging station
    place_id = place['place_id']

    # Use the place function to get the overview data
    overview = client.place(place_id=place_id)['result']
    for item in results['results']:
      if 'aspects' in overview.keys():
        print(overview['result']['aspects'])
    # Check if the charging station has reviews
      if 'reviews' in overview.keys():
        temp_review = overview['reviews']
        for j in range(len(temp_review)):
          if len(temp_review[j]['text'])>1:
            dt_object = datetime.datetime.fromtimestamp(temp_review[j]['time'])

          # Append the overview and review data to the list
          data.append({
            'name': overview['name'],
            'business_status': overview['business_status'],
            'address': overview['formatted_address'],
            'lat': overview['geometry']['location']['lat'],
            'lng': overview['geometry']['location']['lng'],
            'types': ', '.join(overview['types']),
            'rating': overview['rating'],
            'user_ratings_total': overview['user_ratings_total'],
            '#charging_stations': len(temp_review),
            'text': temp_review[j]['text'],
            'author_name': temp_review[j]['author_name'],
            'author_url': temp_review[j]['author_url'],
            'rating': temp_review[j]['rating'],
            'relative_time_description': temp_review[j]['relative_time_description'],
            'time': dt_object.strftime('%Y-%m-%d %H:%M:%S')
                                                                })


ValueError: Invalid API key provided.

In [16]:
params = {
    # https://docs.python.org/3/library/os.html#os.getenv
    'api_key': serp_api_key, # your serpapi api
    'engine': 'google_maps',                # SerpApi search engine	
    'q': 'charge',                          # query
    'll': '@40.7455096,-74.0083012,15.1z',  # GPS coordinates
    'type': 'search',                       # list of results for the query
    'hl': 'en',                             # language
    'start': 0,                             # pagination
}

search = GoogleSearch(params)               # where data extraction happens on the backend
results = search.get_dict()                 # JSON -> Python dict

In [17]:
results['local_results']

[{'position': 1,
  'title': 'Livingston Charge Port Charging Station',
  'place_id': 'ChIJJy6tV61ZwokRYS3DaUMjmKY',
  'data_id': '0x89c259ad57ad2e27:0xa698234369c32d61',
  'data_cid': '12004383579200236897',
  'reviews_link': 'https://serpapi.com/search.json?data_id=0x89c259ad57ad2e27%3A0xa698234369c32d61&engine=google_maps_reviews&hl=en',
  'photos_link': 'https://serpapi.com/search.json?data_id=0x89c259ad57ad2e27%3A0xa698234369c32d61&engine=google_maps_photos&hl=en',
  'gps_coordinates': {'latitude': 40.756279899999996, 'longitude': -73.99461},
  'place_id_search': 'https://serpapi.com/search.json?engine=google_maps&google_domain=google.com&hl=en&place_id=ChIJJy6tV61ZwokRYS3DaUMjmKY',
  'provider_id': '/g/11t7d18w4s',
  'unclaimed_listing': True,
  'type': 'Electric vehicle charging station',
  'types': ['Electric vehicle charging station'],
  'address': '511 9th Ave, New York, NY 10018',
  'phone': '(518) 691-3119',
  'website': 'https://www.solution.energy/',
  'thumbnail': 'https:

In [9]:
local_results_data = []
for result in results['local_results']:
    result_default = defaultdict(lambda : np.NAN, result)
    local_results_data.append((result_default['title'], 
                               result_default['data_id'], 
                                result_default['gps_coordinates'], 
                                result_default['type'], 
                                result_default['address'], 
                                result_default['phone'], 
                                result_default['website'], 
                                result_default['rating'],
                                result_default['reviews'],
     ) )
local_results_data

[('Livingston Charge Port Charging Station',
  '0x89c259ad57ad2e27:0xa698234369c32d61',
  {'latitude': 40.756279899999996, 'longitude': -73.99461},
  'Electric vehicle charging station',
  '511 9th Ave, New York, NY 10018',
  '(518) 691-3119',
  'https://www.solution.energy/',
  nan,
  nan),
 ('Livingston Charge Port Charging Station',
  '0x89c259bacf48b6db:0x48cf36d8cc6e05a',
  {'latitude': 40.745399899999995, 'longitude': -73.99542},
  'Electric vehicle charging station',
  '252 7th Ave, New York, NY 10001',
  '(518) 691-3119',
  'https://www.solution.energy/',
  4,
  1),
 ('Livingston Charge Port Charging Station',
  '0x89c259a39c014cdb:0x47d68867f5654716',
  {'latitude': 40.74057, 'longitude': -73.99146999999999},
  'Electric vehicle charging station',
  '4 W 21st St, New York, NY 10010',
  '(518) 691-3119',
  'https://www.solution.energy/',
  nan,
  nan),
 ('EV Connect Charging Station',
  '0x89c25853281781b7:0x8950f2224138f711',
  {'latitude': 40.756891599999996, 'longitude': -73

In [3]:
review_data = pd.read_csv('../data/Google_Map_review_data.csv')
review_data.head()

,name,business_status,address,lat,lng,types,rating,user_ratings_total,#charging_stations,text,author_name,author_url,relative_time_description,time,Unnamed: 14
0,Electrify America Charging Station,OPERATIONAL,"8300 W Overland Rd, Boise, ID 83709, USA",43.59165,-116.283556,"point_of_interest, establishment",5,9,5,Electric Vehicle Charging\ngood location in Wa...,Anmar Abdullah,https://www.google.com/maps/contrib/1149397606...,7 months ago,2022-09-25 21:33:44,0
1,Electrify America Charging Station,OPERATIONAL,"8300 W Overland Rd, Boise, ID 83709, USA",43.59165,-116.283556,"point_of_interest, establishment",4,9,5,The rates seem a little high and I often find ...,Space Ghost,https://www.google.com/maps/contrib/1005346496...,4 months ago,2023-01-06 2:41:54,0
2,Electrify America Charging Station,OPERATIONAL,"8300 W Overland Rd, Boise, ID 83709, USA",43.59165,-116.283556,"point_of_interest, establishment",1,9,5,Tried 3 different chargers . Still couldn’t ge...,Roxanna Todd,https://www.google.com/maps/contrib/1171099222...,4 months ago,2022-12-18 23:50:11,0
3,Electrify America Charging Station,OPERATIONAL,"8300 W Overland Rd, Boise, ID 83709, USA",43.59165,-116.283556,"point_of_interest, establishment",3,9,5,There's always at least one charger not workin...,Andy Jensen,https://www.google.com/maps/contrib/1112728836...,7 months ago,2022-10-12 22:01:23,0
4,Electrify America Charging Station,OPERATIONAL,"8300 W Overland Rd, Boise, ID 83709, USA",43.59165,-116.283556,"point_of_interest, establishment",2,9,5,Frustrating as hell! Most of them require a 2...,John Hudson,https://www.google.com/maps/contrib/1025132783...,9 months ago,2022-07-24 2:35:54,0


In [5]:
review_data['text'] = review_data['text'].str.replace('\n', ' ')

In [7]:
review_data['text']

0        Electric Vehicle Charging good location in Wal...
1        The rates seem a little high and I often find ...
2        Tried 3 different chargers . Still couldn’t ge...
3        There's always at least one charger not workin...
4        Frustrating as hell!  Most of them require a 2...
                               ...                        
16762                                                  NaN
16763                                                  NaN
16764    Lady cashier is always angry when ever I go th...
16765                                                  NaN
16766                                                  NaN
Name: text, Length: 16767, dtype: object

In [8]:
review_data.columns

Index(['name', 'business_status', 'address', 'lat', 'lng', 'types', 'rating',
       'user_ratings_total', '#charging_stations', 'text', 'author_name',
       'author_url', 'relative_time_description', 'time', 'Unnamed: 14'],
      dtype='object')

In [22]:
def merge_comments_ratings(df):
    return_df = df[['name', 'business_status', 'address', 'types',
       'user_ratings_total', '#charging_stations']].drop_duplicates(keep='first')
    all_comments = ""

    for index, row in df.iterrows():
        all_comments = all_comments + row['text']

    return_df['comments'] = all_comments
    return_df['avg_rating'] = df['rating'].mean()
    return return_df


review_data.dropna(subset=['text'], inplace=True)
agged_review = review_data.groupby(by=['lat', 'lng']).apply(merge_comments_ratings).reset_index()

In [27]:
docid_NREL_map = pd.read_csv('../data/docid_NREL_map.csv')

merged = merge_lat_lng(docid_NREL_map, agged_review)

100%|██████████| 67622/67622 [00:46<00:00, 1461.44it/s]


In [32]:
docid_NREL_map.shape

(67622, 3)

In [33]:
merged.drop_duplicates(subset=['docid'], inplace=True)
merged

,docid,lat,lng,lat,lng,level_2,name,business_status,address,types,user_ratings_total,#charging_stations,comments,avg_rating
0,113,40.119161,-88.244185,40.119124,-88.244590,1778,Electric Vehicle Charging Station,OPERATIONAL,"123 W Hill St, Champaign, IL 61820, USA","point_of_interest, establishment",1,1,Level 2 Chargers available in parking garage o...,5.0
1,122,44.586123,-123.115389,44.586120,-123.115363,7459,ChargePoint Charging Station,OPERATIONAL,"Ellingson Rd SW, Albany, OR 97321, USA","point_of_interest, establishment",4,4,This EV charging station is maintained by Linn...,5.0
2,207,30.206145,-97.759109,30.206084,-97.759519,13511,Nissan Charging Station,OPERATIONAL,"4914 S I-35 Frontage Rd, Austin, TX 78745, USA","point_of_interest, establishment",3,3,Gives +~ 19 miles per hour of charge. Good l...,4.0
3,214,36.010279,-84.268049,36.010202,-84.268102,10666,Electric Vehicle Charging Station,OPERATIONAL,"101 S Illinois Ave, Oak Ridge, TN 37830, USA","point_of_interest, establishment",4,4,Great for electric power cars. Enough to useCo...,4.5
4,221,30.062535,-94.133511,30.062232,-94.133716,1481,Nissan Charging Station,OPERATIONAL,"1515 I-10 S, Beaumont, TX 77701, USA","point_of_interest, establishment",19,5,"Really useful, reasonably fast charging statio...",5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6088,67583,35.400260,-119.397501,35.400148,-119.397794,8622,Tesla Supercharger,OPERATIONAL,"20673 Tracy Ave, Buttonwillow, CA 93206, USA","point_of_interest, establishment",91,5,Small detour from I5. Close to a lot of food p...,3.8
6089,67584,35.400260,-119.397501,35.400148,-119.397794,8622,Tesla Supercharger,OPERATIONAL,"20673 Tracy Ave, Buttonwillow, CA 93206, USA","point_of_interest, establishment",91,5,Small detour from I5. Close to a lot of food p...,3.8
6090,67585,35.400260,-119.397501,35.400148,-119.397794,8622,Tesla Supercharger,OPERATIONAL,"20673 Tracy Ave, Buttonwillow, CA 93206, USA","point_of_interest, establishment",91,5,Small detour from I5. Close to a lot of food p...,3.8
6091,67586,35.400260,-119.397501,35.400148,-119.397794,8622,Tesla Supercharger,OPERATIONAL,"20673 Tracy Ave, Buttonwillow, CA 93206, USA","point_of_interest, establishment",91,5,Small detour from I5. Close to a lot of food p...,3.8


In [25]:
agged_review.to_csv('../data/Google_Map_review_data_aggregated.csv', index=False)